In [1]:
import wandb
from lightning.pytorch.callbacks import Callback
import lightning as L
from model.vit import ViT
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from data.dataset import Im2LatexDataset
from tqdm.auto import tqdm
from utils.inkml2img import convert_dir
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
import torch
import sys
sys.path.append("../")

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_dims = [224, 640]
model = ViT(img_dims, 16, n_embd=512, encoder=False, output_classes=10)
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, classification=True, batch_size=16)
batch = next(iter(data.train))

In [3]:
model = ViT(img_dims, 16, n_embd=512, encoder=False, output_classes=1)
model.training_step(batch, 1)

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/core/module.py:407: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


tensor(0.5773, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [4]:
# checkpoint_callback = ModelCheckpoint(monitor='train accuracy', mode='max')
logger = WandbLogger(project='vittest')


class LogPredictionsCallback(Callback):
    def on_validation_batch_end(
            self, trainer, pl_module, outputs, batch, batch_idx):
        """Called when the validation batch ends."""

        # `outputs` comes from `LightningModule.validation_step`
        # which corresponds to our model predictions in this case

        # Let's log 20 sample image predictions from first batch
        if batch_idx == 0:
            n = 20
            x, y = batch
            images = [img for img in x[:n]]
            captions = [
                f'Ground Truth: {y_i} - Prediction: {y_pred}' for y_i, y_pred in zip(y[:n], outputs[:n])]

            # Option 1: log images with `WandbLogger.log_image`
            logger.log_image(key='sample_images',
                             images=images, caption=captions)

            # Option 2: log predictions as a Table
            columns = ['image', 'ground truth', 'prediction']
            data = [[wandb.Image(x_i), y_i, y_pred] for x_i, y_i, y_pred in list(
                zip(x[:n], y[:n], outputs[:n]))]
            logger.log_table(key='sample_table', columns=columns, data=data)


log_predictions_callback = LogPredictionsCallback()
trainer = L.Trainer(limit_train_batches=16, max_epochs=20,
                    logger=logger, callbacks=[log_predictions_callback])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: baolong. Use `wandb login --relogin` to force relogin


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, data.train, data.val)
wandb.finish()


  | Name               | Type            | Params
-------------------------------------------------------
0 | patch_embeddings   | PatchEmbeddings | 131 K 
1 | pos_dropout        | Dropout         | 0     
2 | transformer_blocks | ModuleList      | 18.9 M
3 | ln                 | LayerNorm       | 1.0 K 
4 | head               | Linear          | 513   
-------------------------------------------------------
19.3 M    Trainable params
0         Non-trainable params
19.3 M    Total params
77.279    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s] 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/torchmetrics/metric.py:390 in │
│ wrapped_func                                                                                     │
│                                                                                                  │
│   387 │   │   │   self._update_count += 1                                                        │
│   388 │   │   │   with torch.set_grad_enabled(self._enable_grad):                                │
│   389 │   │   │   │   try:                                                                       │
│ ❱ 390 │   │   │   │   │   update(*args, **kwargs)                                                │
│   391 │   │   │   │   except RuntimeError as err:                                                │
│   392 │   │   │   │   │   if "Expected all tensors to be on" in str(err):                        │
│   393 │   │   │   │   │   │   raise RuntimeError(                                                │
│                                                                                                  │
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/torchmetrics/classification/s │
│ tat_scores.py:178 in update                                                                      │
│                                                                                                  │
│   175 │   │   │   _binary_stat_scores_tensor_validation(preds, target, self.multidim_average,    │
│   176 │   │   preds, target = _binary_stat_scores_format(preds, target, self.threshold, self.i   │
│   177 │   │   tp, fp, tn, fn = _binary_stat_scores_update(preds, target, self.multidim_average   │
│ ❱ 178 │   │   self._update_state(tp, fp, tn, fn)                                                 │
│   179 │                                                                                          │
│   180 │   def compute(self) -> Tensor:                                                           │
│   181 │   │   """Computes the final statistics."""                                               │
│                                                                                                  │
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/torchmetrics/classification/s │
│ tat_scores.py:70 in _update_state                                                                │
│                                                                                                  │
│    67 │   │   │   self.tn.append(tn)                                                             │
│    68 │   │   │   self.fn.append(fn)                                                             │
│    69 │   │   else:                                                                              │
│ ❱  70 │   │   │   self.tp += tp                                                                  │
│    71 │   │   │   self.fp += fp                                                                  │
│    72 │   │   │   self.tn += tn                                                                  │
│    73 │   │   │   self.fn += fn                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(model, data.train, data.val)                                      